# LANDARTs - Landsat automatic retrieval of surface temperatures

LANDARTs is a Python tool that works (currently) only with Landsat archives in the new metadata file format (data acquired after 2012) that can be downloaded from the [USGS website](https://earthexplorer.usgs.gov/).

In [11]:
import glob, subprocess, os
import rasterio
import geopandas as gpd
from pathlib import Path

**Set directory**

In [2]:
computer_path = '/export/miro/ndeffense/LBRAT2104/'
grp_nb        = '99'

# Directory for all work files
work_path = f'{computer_path}STUDENTS/GROUP_{grp_nb}/TP/'

# Input directory
landsat_path = f'{work_path}LANDSAT/'

# Output directory
lst_path = f'{work_path}LST/'

Path(lst_path).mkdir(parents=True, exist_ok=True)

## Launch LANDARTs to get LST

```
To be launch on the lab'servers - LANDARTs tool can not be installed in computer room !
```

In [9]:
for input_file in glob.glob(f'{landsat_path}*.tar.gz'):
    
    print(input_file)
    
    output_file = f'{lst_path}{os.path.basename(input_file)[:-7]}_LST.tif'
    
    if not os.path.isfile(output_file):
        
        cmd = '/export/apps/os152/pyboxes/bin/geomatics.prepro.landarts.pb'
        cmd += f' {input_file}'
        cmd += f' {output_file}'
        
        print(cmd)
        #subprocess.call(cmd, shell=True)

/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/LANDSAT/LC08_L1TP_197025_20180804_20180814_01_T1.tar.gz
/export/apps/os152/pyboxes/bin/geomatics.prepro.landarts.pb /export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/LANDSAT/LC08_L1TP_197025_20180804_20180814_01_T1.tar.gz /export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/LST/LC08_L1TP_197025_20180804_20180814_01_T1_LST.tif


Traceback (most recent call last):
  File "/export/apps/os152/pyboxes/bin/../lib/python3.6/site-packages/pb-geomatics/prepro/landarts.pb.py", line 2, in <module>
    import pybox
ModuleNotFoundError: No module named 'pybox'


## Clip to ROI

## Convert Farenheit to Celcius

In [10]:
for input_file in glob.glob(f'{lst_path}*_LST.tif'):
    
    print(input_file)

    output_file = f'{lst_path}{os.path.basename(input_file)[:-4]}_LSTinC.tif'

    data_type = ' Float32'

    if not os.path.isfile(output_file):
            
        cmd = 'gdal_calc.py'
        cmd += ' -A ' + input_file
        cmd += ' --outfile=' + output_file
        cmd += ' --type' + data_type
        cmd += ' --calc=' + '"1.0*A/100.0 - 273.15"'

        print(cmd)
        subprocess.call(cmd, shell=True)


/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/LST/LC08_L1TP_197025_20180804_20180814_01_T1_LST.tif
gdal_calc.py -A /export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/LST/LC08_L1TP_197025_20180804_20180814_01_T1_LST.tif --outfile=/export/miro/ndeffense/LBRAT2104/STUDENTS/GROUP_99/TP/LST/LC08_L1TP_197025_20180804_20180814_01_T1_LST_LSTinC.tif --type Float32 --calc="1.0*A/100.0 - 273.15"
0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1

## Apply Cloud Mask

In [ ]:
out_dir = '/export/homes/students/abos/images/CM/'

for f in glob.glob('/export/homes/students/abos/images/BQA/*BQA.TIF'):
    directory, archiveName = os.path.split(f)
    out = out_dir + archiveName[:-4] + '_CM.tif'
    print(out)

    data_type = ' UInt16'

    if not os.path.isfile(out):

        cmd = 'gdal_calc.py'
        cmd += ' -F ' + f
        cmd += ' --outfile=' + out
        cmd += ' --type' + data_type
        cmd += ' --calc=' + '"(F==2720)*1"'
        cmd += ' --NoDataValue=0'

        print(cmd)
        subprocess.run(cmd,shell=True)